In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

In [2]:
train_data = pd.read_csv('dataset/Google_Stock_Train.csv')
train_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,15.689439,15.753504,15.621622,15.684434,15.684434,78169752
1,2010-01-05,15.695195,15.711712,15.554054,15.615365,15.615365,120067812
2,2010-01-06,15.662162,15.662162,15.174174,15.221722,15.221722,158988852
3,2010-01-07,15.250250,15.265265,14.831081,14.867367,14.867367,256315428
4,2010-01-08,14.814815,15.096346,14.742492,15.065566,15.065566,188783028
...,...,...,...,...,...,...,...
3267,2022-12-23,87.110001,89.550003,87.070000,89.230003,89.230003,23003000
3268,2022-12-27,88.800003,88.940002,87.010002,87.389999,87.389999,20097300
3269,2022-12-28,86.980003,88.040001,85.940002,86.019997,86.019997,19523200
3270,2022-12-29,86.620003,88.849998,86.610001,88.449997,88.449997,23333500


In [3]:
test_data = pd.read_csv('dataset/Google_Stock_Test.csv')
test_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-03,89.589996,91.050003,88.519997,89.120003,89.120003,28131200
1,2023-01-04,90.349998,90.650002,87.269997,88.080002,88.080002,34854800
2,2023-01-05,87.470001,87.570000,85.900002,86.199997,86.199997,27194400
3,2023-01-06,86.790001,87.690002,84.860001,87.339996,87.339996,41381500
4,2023-01-09,88.360001,90.050003,87.860001,88.019997,88.019997,29003900
...,...,...,...,...,...,...,...
138,2023-07-24,121.660004,123.000000,120.980003,121.529999,121.529999,29686100
139,2023-07-25,121.360001,123.150002,121.019997,122.209999,122.209999,52509600
140,2023-07-26,130.070007,130.979996,128.320007,129.270004,129.270004,61682100
141,2023-07-27,131.669998,133.240005,128.789993,129.399994,129.399994,44952100


In [4]:
full_data = pd.concat([train_data, test_data], ignore_index=True)
full_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-01-04,15.689439,15.753504,15.621622,15.684434,15.684434,78169752
1,2010-01-05,15.695195,15.711712,15.554054,15.615365,15.615365,120067812
2,2010-01-06,15.662162,15.662162,15.174174,15.221722,15.221722,158988852
3,2010-01-07,15.250250,15.265265,14.831081,14.867367,14.867367,256315428
4,2010-01-08,14.814815,15.096346,14.742492,15.065566,15.065566,188783028
...,...,...,...,...,...,...,...
3410,2023-07-24,121.660004,123.000000,120.980003,121.529999,121.529999,29686100
3411,2023-07-25,121.360001,123.150002,121.019997,122.209999,122.209999,52509600
3412,2023-07-26,130.070007,130.979996,128.320007,129.270004,129.270004,61682100
3413,2023-07-27,131.669998,133.240005,128.789993,129.399994,129.399994,44952100


In [5]:
minmax = MinMaxScaler().fit(full_data.iloc[:, 4:5].astype('float32'))

In [6]:
train_data = minmax.transform(train_data.iloc[:, 4:5]).astype('float32')
train_data = pd.DataFrame(train_data)
train_data

,0
0,0.034348
1,0.033850
2,0.031017
3,0.028466
4,0.029893
...,...
3267,0.563735
3268,0.550490
3269,0.540629
3270,0.558120


In [7]:
test_data = minmax.transform(test_data.iloc[:, 4:5]).astype('float32')
test_data = pd.DataFrame(test_data)
test_data

,0
0,0.562943
1,0.555457
2,0.541925
3,0.550130
4,0.555025
...,...
138,0.796233
139,0.801128
140,0.851946
141,0.852882


# Functions

In [36]:
def forecast(model):
    modelnn = model
    date_ori = pd.to_datetime(full_data.iloc[:, 0]).tolist()

    pbar = tqdm(range(epoch), desc = 'train loop')
    for i in pbar:
        total_loss, total_acc = [], []
        for k in range(0, train_data.shape[0] - 1, timestamp):
            index = min(k + timestamp, train_data.shape[0] - 1)
            batch_x = np.expand_dims(
                train_data.iloc[k : index, :].values, axis = 0
            )
            batch_y = train_data.iloc[k + 1 : index + 1, :].values
            
            with tf.GradientTape() as tape:
                logits = modelnn(batch_x)
                loss = modelnn.compute_loss(batch_y, logits)
            
            gradients = tape.gradient(loss, modelnn.trainable_variables)
            modelnn.optimizer.apply_gradients(zip(gradients, modelnn.trainable_variables))
            
            total_loss.append(loss.numpy())
            total_acc.append(modelnn.compute_accuracy(batch_y[:, 0], logits[:, 0]))
        pbar.set_postfix(cost = np.mean(total_loss), acc = np.mean(total_acc))

    future_day = test_size
    output_predict = np.zeros((full_data_train.shape[0] + future_day, full_data_train.shape[1]))
    output_predict[0] = full_data_train.iloc[0]

    for k in range(0, full_data_train.shape[0], timestamp):
        index = min(k + timestamp, full_data_train.shape[0])
        batch_x = np.expand_dims(full_data_train.iloc[k:index, :].values, axis=0)
        out_logits = modelnn(batch_x).numpy()
        output_predict[k + 1:index + 1] = out_logits

    for i in range(future_day):
        o = output_predict[-future_day - timestamp + i:-future_day + i]
        batch_x = np.expand_dims(o, axis=0)
        out_logits = modelnn(batch_x).numpy()
        output_predict[-future_day + i] = out_logits[-1]
        date_ori.append(date_ori[-1] + timedelta(days=1))

    output_predict = minmax.inverse_transform(output_predict)
    deep_future = anchor(output_predict[:, 0], 0.3)

    return deep_future[-test_size:]

In [ ]:
def visualize_accuracy(results):
    accuracies = [calculate_accuracy(full_data['Close'].iloc[-test_size:].values, r) for r in results]

    plt.figure(figsize = (15, 5))
    for no, r in enumerate(results):
        plt.plot(r, label = 'forecast %d'%(no + 1))
    plt.plot(full_data['Close'].iloc[-test_size:].values, label = 'true trend', c = 'black')
    plt.legend()
    plt.title('average accuracy: %.4f'%(np.mean(accuracies)))
    plt.show()

# LSTM

In [37]:
class LstmModel(tf.keras.Model):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        super(LstmModel, self).__init__()
        
        self.lstm_layers = [
            tf.keras.layers.LSTM(
                size_layer,
                return_sequences=True,
                dropout=forget_bias,
            ) for _ in range(num_layers)
        ]
    
        self.dense = tf.keras.layers.Dense(output_size)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

    def call(self, inputs):
        x = inputs
        for lstm_layer in self.lstm_layers:
            x = lstm_layer(x)
        
        logits = self.dense(x[:, -1, :])
        return logits

    def compute_loss(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_true - y_pred))

    def compute_accuracy(self, y_true, y_pred):
        real = np.array(y_true) + 1
        predict = np.array(y_pred) + 1
        percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
        return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [38]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 300
dropout_rate = 0.8
future_day = test_data.shape[0]
learning_rate = 0.01
simulation_size = 10
test_size = test_data.shape[0]

lstm = LstmModel(
    learning_rate, 
    num_layers, 
    full_data.shape[1], 
    size_layer, 
    full_data.shape[1], 
    dropout_rate
    )

In [ ]:
results = []
for i in range(simulation_size):
    print('simulation %d'%(i + 1))
    results.append(forecast(lstm))

simulation 1


train loop:  48%|████▊     | 144/300 [46:03<1:08:04, 26.18s/it, acc=99.1, cost=0.000531]

In [ ]:
visualize_accuracy(results)

# Bidirectional LSTM

In [ ]:
class BiLstmModel(tf.keras.Model):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        super(BiLstmModel, self).__init__()
        
        self.lstm_layers = [
            tf.keras.layers.LSTM(
                size_layer,
                return_sequences=True,
                dropout=forget_bias,
            ) for _ in range(num_layers)
        ]

        self.bidirectional_lstm = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(size_layer, return_sequences=True)
        )
    
        self.dense = tf.keras.layers.Dense(output_size)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

    def call(self, inputs):
        x = inputs
        for lstm_layer in self.lstm_layers:
            x = lstm_layer(x)

        x = self.bidirectional_lstm(x)
        
        logits = self.dense(x[:, -1, :])
        return logits

    def compute_loss(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_true - y_pred))

    def compute_accuracy(self, y_true, y_pred):
        real = np.array(y_true) + 1
        predict = np.array(y_pred) + 1
        percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
        return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [ ]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 300
dropout_rate = 0.8
future_day = test_data.shape[0]
learning_rate = 0.01
simulation_size = 10
test_size = test_data.shape[0]

bilstm = BiLstmModel(
    learning_rate, 
    num_layers, 
    full_data.shape[1], 
    size_layer, 
    full_data.shape[1], 
    dropout_rate
    )

In [ ]:
results = []
for i in range(simulation_size):
    print('simulation %d'%(i + 1))
    results.append(forecast(bilstm))

In [ ]:
visualize_accuracy(results)

# GRU

In [ ]:
class GruModel(tf.keras.Model):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        forget_bias = 0.1,
    ):
        super(GruModel, self).__init__()
        
        self.gru_layers = [
            tf.keras.layers.GRU(size_layer, return_sequences=True, dropout=forget_bias)
            for _ in range(num_layers)
        ]
    
        self.dense = tf.keras.layers.Dense(output_size)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

    def call(self, inputs):
        x = inputs
        for gru_layer  in self.gru_layer :
            x = gru_layer (x)

        logits = self.dense(x[:, -1, :])
        return logits

    def compute_loss(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_true - y_pred))

    def compute_accuracy(self, y_true, y_pred):
        real = np.array(y_true) + 1
        predict = np.array(y_pred) + 1
        percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
        return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [ ]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 300
dropout_rate = 0.8
future_day = test_data.shape[0]
learning_rate = 0.01
simulation_size = 10
test_size = test_data.shape[0]

gru = GruModel(
    learning_rate, 
    num_layers, 
    full_data.shape[1], 
    size_layer, 
    full_data.shape[1], 
    dropout_rate
    )

In [ ]:
results = []
for i in range(simulation_size):
    print('simulation %d'%(i + 1))
    results.append(forecast(gru))

In [ ]:
visualize_accuracy(results)

# Attention

In [ ]:
def layer_norm(inputs, epsilon=1e-8):
    norm_layer = tf.keras.layers.LayerNormalization(epsilon=epsilon)
    return norm_layer(inputs)

def multihead_attn(queries, keys, q_masks, k_masks, future_binding, num_units, num_heads):
    T_q = tf.shape(queries)[1]                                      
    T_k = tf.shape(keys)[1]                  

    Q = tf.keras.layers.Dense(num_units, name='Q')(queries)                              
    K_V = tf.keras.layers.Dense(2 * num_units, name='K_V')(keys)    
    K, V = tf.split(K_V, 2, -1)        

    Q_ = tf.concat(tf.split(Q, num_heads, axis=2), axis=0)                         
    K_ = tf.concat(tf.split(K, num_heads, axis=2), axis=0)                    
    V_ = tf.concat(tf.split(V, num_heads, axis=2), axis=0)                      

    align = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1]))                      
    align = align / tf.sqrt(tf.cast(K_.shape[-1], tf.float32))

    paddings = tf.fill(tf.shape(align), float('-inf'))                   

    key_masks = k_masks                                                 
    key_masks = tf.tile(key_masks, [num_heads, 1])                       
    key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, T_q, 1])            
    align = tf.where(tf.equal(key_masks, 0), paddings, align)       

    if future_binding:
        lower_tri = tf.ones([T_q, T_k])                                          
        lower_tri = tf.linalg.LinearOperatorLowerTriangular(lower_tri).to_dense()  
        masks = tf.tile(tf.expand_dims(lower_tri, 0), [tf.shape(align)[0], 1, 1]) 
        align = tf.where(tf.equal(masks, 0), paddings, align)                      
    
    align = tf.nn.softmax(align)                                            
    query_masks = tf.cast(q_masks, tf.float32)  # Replaced `tf.to_float`
    query_masks = tf.tile(query_masks, [num_heads, 1])                             
    query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, T_k])            
    align *= query_masks
    
    outputs = tf.matmul(align, V_)                                                 
    outputs = tf.concat(tf.split(outputs, num_heads, axis=0), axis=2)             
    outputs += queries                                                             
    outputs = layer_norm(outputs)                                                 
    return outputs

def pointwise_feedforward(inputs, hidden_units, activation=None):
    outputs = tf.keras.layers.Dense(4 * hidden_units, activation=activation)(inputs)
    outputs = tf.keras.layers.Dense(hidden_units, activation=None)(outputs)
    outputs += inputs
    outputs = layer_norm(outputs)
    return outputs

def sinusoidal_position_encoding(inputs, mask, repr_dim):
    T = tf.shape(inputs)[1]
    pos = tf.reshape(tf.range(0.0, tf.cast(T, tf.float32), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1]) * tf.expand_dims(tf.cast(mask, tf.float32), -1)

def label_smoothing(inputs, epsilon=0.1):
    C = inputs.shape[-1]
    return ((1 - epsilon) * inputs) + (epsilon / C)

class Attention(tf.keras.Model):
    def __init__(self, size_layer, embedded_size, learning_rate, size, output_size,
                 num_blocks=2, num_heads=8, min_freq=50):
        super(Attention, self).__init__()

        self.X = tf.keras.Input(shape=(None, size))  # Replaced placeholder with Keras Input
        self.Y = tf.keras.Input(shape=(output_size,))

        # Define the encoder embedding
        encoder_embedded = tf.keras.layers.Dense(embedded_size)(self.X)
        encoder_embedded = tf.keras.layers.Dropout(0.2)(encoder_embedded)
        x_mean = tf.reduce_mean(self.X, axis=2)
        en_masks = tf.sign(x_mean)
        encoder_embedded += sinusoidal_position_encoding(self.X, en_masks, embedded_size)
        
        for i in range(num_blocks):
            with tf.name_scope(f'encoder_self_attn_{i}'):
                encoder_embedded = multihead_attn(queries=encoder_embedded,
                                                 keys=encoder_embedded,
                                                 q_masks=en_masks,
                                                 k_masks=en_masks,
                                                 future_binding=False,
                                                 num_units=size_layer,
                                                 num_heads=num_heads)

            with tf.name_scope(f'encoder_feedforward_{i}'):
                encoder_embedded = pointwise_feedforward(encoder_embedded,
                                                         embedded_size,
                                                         activation=tf.nn.relu)
                
        self.logits = tf.keras.layers.Dense(output_size)(encoder_embedded)
        self.cost = tf.reduce_mean(tf.square(self.Y - self.logits))
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

    def call(self, inputs):
        return self(inputs)

    def compute_loss(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_true - y_pred))

    def compute_accuracy(self, y_true, y_pred):
        real = np.array(y_true) + 1
        predict = np.array(y_pred) + 1
        percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
        return percentage * 100

In [ ]:
num_layers = 1
size_layer = 128
timestamp = 5
epoch = 300
dropout_rate = 0.8
future_day = test_data.shape[0]
learning_rate = 0.01
simulation_size = 10
test_size = test_data.shape[0]

attention = Attention(
    learning_rate, 
    num_layers, 
    full_data.shape[1], 
    size_layer, 
    full_data.shape[1], 
    dropout_rate
    )

In [ ]:
results = []
for i in range(simulation_size):
    print('simulation %d'%(i + 1))
    results.append(forecast(attention))

In [ ]:
visualize_accuracy(results)

# Dilated CNN

In [ ]:
import tensorflow as tf
import numpy as np

def position_encoding(inputs):
    T = tf.shape(inputs)[1]
    repr_dim = inputs.shape[-1]
    pos = tf.reshape(tf.range(0.0, tf.cast(T, tf.float32), dtype=tf.float32), [-1, 1])
    i = np.arange(0, repr_dim, 2, np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, [-1], keepdims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.shape[-1:]
    gamma = tf.Variable(tf.ones(params_shape), name='gamma')
    beta = tf.Variable(tf.zeros(params_shape), name='beta')
    return gamma * normalized + beta

def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x = tf.keras.layers.Conv1D(filters=hidden_dim,
                              kernel_size=kernel_size,
                              dilation_rate=dilation_rate,
                              padding='valid')(tf.concat([pad, x, pad], 1))
    x = x[:, :-pad_sz, :]
    x = tf.nn.relu(x)
    return x

class CnnModel(tf.keras.Model):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        kernel_size=3,
        n_attn_heads=16,
        dropout=0.9,
    ):
        super(CnnModel, self).__init__()
        self.num_layers = num_layers
        self.size_layer = size_layer
        self.kernel_size = kernel_size
        self.n_attn_heads = n_attn_heads
        self.dropout = dropout
        self.dense_encoder = tf.keras.layers.Dense(size_layer)
        self.conv_layers = []
        self.attn_conv_layers = []
        self.attn_dense_h = []
        self.attn_dense_g = []
        self.attn_dense_zu = []
        self.attn_dense_ze = []
        self.attn_dense_d = []
        self.attn_dense_h_combined = []
        self.dense_output = tf.keras.layers.Dense(output_size)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate)

        for i in range(num_layers):
            self.conv_layers.append(tf.keras.layers.Conv1D(filters=size_layer, kernel_size=kernel_size, dilation_rate=2**i, padding='valid'))
            self.attn_conv_layers.append(tf.keras.layers.Conv1D(filters=size_layer, kernel_size=kernel_size, dilation_rate=2**i, padding='valid'))
            for j in range(n_attn_heads):
                self.attn_dense_h.append(tf.keras.layers.Dense(size_layer // n_attn_heads))
                self.attn_dense_g.append(tf.keras.layers.Dense(size_layer // n_attn_heads))
                self.attn_dense_zu.append(tf.keras.layers.Dense(size_layer // n_attn_heads))
                self.attn_dense_ze.append(tf.keras.layers.Dense(size_layer // n_attn_heads))
                self.attn_dense_d.append(tf.keras.layers.Dense(size_layer // n_attn_heads))
            self.attn_dense_h_combined.append(tf.keras.layers.Dense(size_layer))

    def call(self, inputs):
        encoder_embedded = self.dense_encoder(inputs)
        encoder_embedded += position_encoding(encoder_embedded)

        e = tf.identity(encoder_embedded)
        for i in range(self.num_layers):
            dilation_rate = 2 ** i
            pad_sz = (self.kernel_size - 1) * dilation_rate
            with tf.name_scope(f'block_{i}'):
                encoder_embedded += cnn_block(encoder_embedded, dilation_rate, pad_sz, self.size_layer, self.kernel_size)

        encoder_output, output_memory = encoder_embedded, encoder_embedded + e
        g = tf.identity(encoder_embedded)

        for i in range(self.num_layers):
            dilation_rate = 2 ** i
            pad_sz = (self.kernel_size - 1) * dilation_rate
            with tf.name_scope(f'decode_{i}'):
                attn_res = h = cnn_block(encoder_embedded, dilation_rate, pad_sz, self.size_layer, self.kernel_size)

            C = []
            for j in range(self.n_attn_heads):
                h_ = self.attn_dense_h[i * self.n_attn_heads + j](h)
                g_ = self.attn_dense_g[i * self.n_attn_heads + j](g)
                zu_ = self.attn_dense_zu[i * self.n_attn_heads + j](encoder_output)
                ze_ = self.attn_dense_ze[i * self.n_attn_heads + j](output_memory)

                d = self.attn_dense_d[i * self.n_attn_heads + j](h_) + g_
                dz = tf.matmul(d, tf.transpose(zu_, [0, 2, 1]))
                a = tf.nn.softmax(dz)
                c_ = tf.matmul(a, ze_)
                C.append(c_)

            c = tf.concat(C, 2)
            h = self.attn_dense_h_combined[i](attn_res + c)
            h = tf.nn.dropout(h, rate=1-self.dropout)
            encoder_embedded += h

        encoder_embedded = tf.sigmoid(encoder_embedded[:, -1, :])
        logits = self.dense_output(encoder_embedded)
        return logits

    def compute_loss(self, y_true, y_pred):
        return tf.reduce_mean(tf.square(y_true - y_pred))

    def compute_accuracy(self, y_true, y_pred):
        real = np.array(y_true) + 1
        predict = np.array(y_pred) + 1
        percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
        return percentage * 100

def calculate_accuracy(real, predict):
    real = np.array(real) + 1
    predict = np.array(predict) + 1
    percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
    return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [ ]:
num_layers = 1
size_layer = 128
timestamp = test_size
epoch = 300
dropout_rate = 0.8
future_day = test_size
learning_rate = 5e-4

cnn = CnnModel(
        learning_rate, num_layers, full_data.shape[1], size_layer, full_data.shape[1], 
        dropout = dropout_rate
    )

In [ ]:
results = []
for i in range(simulation_size):
    print('simulation %d'%(i + 1))
    results.append(forecast(cnn))

In [ ]:
visualize_accuracy(results)